### OBTENCION DE DATOS API MERCADOLIBRE

#### Importacion de librerias necesarias

In [1]:
import json
import requests
from urllib.parse import quote
import pandas as pd

#### Obtencion de categorias

El objetivo de esta seccion es obtener un listado de las categorias y sub-categorias disponibles actualmente en la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>Como primer paso se hace una request a la API para obtener el listado de las categorias principales.  

In [2]:
#REQUEST PARA OBTENER LAS CATEGORIAS PRINCIPALES
url = "https://api.mercadolibre.com/sites/MLA/categories"
response = requests.get(url)

De la respuesta obtenida se extraen los los ID de las categorias principales. El objetivo de esto es hacer una request por cada ID de categoria pricipal y asi obtener los datos completos de estas, incluidas las subcategorias.

In [3]:
#OBTENCION DE LOS ID DE LAS CATEGORIAS PRINCIPALES
if response.status_code == 200:
    id_list = [item['id'] for item in response.json()]    
else:
    print(response.status_code)
print(id_list)

['MLA5725', 'MLA1512', 'MLA1403', 'MLA1071', 'MLA1367', 'MLA1368', 'MLA1743', 'MLA1384', 'MLA1246', 'MLA1039', 'MLA1051', 'MLA1648', 'MLA1144', 'MLA1500', 'MLA1276', 'MLA5726', 'MLA1000', 'MLA2547', 'MLA407134', 'MLA1574', 'MLA1499', 'MLA1459', 'MLA1182', 'MLA3937', 'MLA1132', 'MLA3025', 'MLA1168', 'MLA1430', 'MLA409431', 'MLA1540', 'MLA9304', 'MLA1953']


Como se menciono anteriormente se generea una request para cada "id" de la lista **id_list** y se alojan los resultados obtenidos en la lista **categories**. El resultado obtenido es una lista de diccionarios.

In [4]:
#OBTENCION DE INFORMACION Y SUBCATEGORIAS DE LAS CATEGORIAS POR SU ID  
categories = []

for id in id_list:
    response = requests.get(f"https://api.mercadolibre.com/categories/{id}")
    if response.status_code == 200:
        categories.append(response.json())

Se crean dos funciones para obtener los datos datos de interes de la lista de categorias y sus respectivas categorias hijas. 

In [5]:
#FUNCION PARA PARA TRAER INFORMACION DE TODAS LAS CATEGORIAS
def get_categories(data:list) -> list:
    categories = []        
    for item in data:
        id = item.get('id', None)
        category = {
                'id': id,
                'name': item.get('name', None) ,
                'is_subcategoy': False,
                'father_cat': None
            }
        categories.append(category)
        if 'children_categories' in item:
            categories.extend(get_sub_categories(item['children_categories'], id))            
    return categories


#FUNCION PARA RETORNAR LAS SUBCATEGORIAS
def get_sub_categories(sub_categories:list, father_cat_id:str) -> list:
    sub_category_list = []
    for sub_cat in sub_categories:
        sub_category_list.append( 
            {
                'id': sub_cat.get('id', None),
                'name': sub_cat.get('name', None),
                'is_subcategoy': True,
                'father_cat': father_cat_id
            }
        )       
    return sub_category_list 

Haciendo uso de las funciones creadas anteriormente se extraen los datos de interes y se los aloja en la lista **categories**, en formato de diccionario.

In [6]:
#OBTENCION DE DATOS DE INTERES DE CATEGORIAS Y SUBCATEGORIAS
categories = get_categories(categories)

Se guarda el archivo **JSON** de manera local

In [7]:
#SE GUARDA LOS DATOS EN UN ARCHIVO JSON
with open('MeliCategorias.json', 'w') as file:
    json.dump(categories, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**categories**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [8]:
df = pd.DataFrame(categories)
df.head()

,id,name,is_subcategoy,father_cat
0,MLA5725,Accesorios para Vehículos,False,None
1,MLA4711,Acc. para Motos y Cuatriciclos,True,MLA5725
2,MLA417044,Accesorios Náuticos,True,MLA5725
3,MLA6520,Accesorios de Auto y Camioneta,True,MLA5725
4,MLA86360,Accesorios para Línea Pesada,True,MLA5725


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [9]:
df = pd.read_json('MeliCategorias.json')
df.head()

,id,name,is_subcategoy,father_cat
0,MLA5725,Accesorios para Vehículos,False,None
1,MLA4711,Acc. para Motos y Cuatriciclos,True,MLA5725
2,MLA417044,Accesorios Náuticos,True,MLA5725
3,MLA6520,Accesorios de Auto y Camioneta,True,MLA5725
4,MLA86360,Accesorios para Línea Pesada,True,MLA5725


#### Obtencion de publicaciones

El objetivo de esta seccion es obtener un listado de publicaciones de "vehiculos" desde la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>En primer lugar se crea una lista(**cars**) que aloja diferentes modelos de vehiculos disponibles en el mercado automotriz, posteriormente mediante el uso de la libreria **urllib** y su metodo **quote** se codifica la lista **cars** a fin de transformar ciertos caracteres para mantener la integridad de la URL al momento de realizar la request.

In [10]:
#LISTA DE MODELOS DE VEHICULOS
cars = [    'Volkswagen gol','Volkswagen gol trend', 'Volkswagen up', 'Volkswagen bora', 'Volkswagen vento', 'Volkswagen polo', 'Volkswagen amarok',
            'Chevrolet corsa', 'Chevrolet onix', 'Chevrolet agile', 'Chevrolet prisma', 'Chevrolet cruze', 'Chevrolet aveo', 'Chevrolet tracker',
            'Renault clio', 'Renault sandero', 'Renault fluence', 'Renault logan', 'Reanault partner', 'Renault capture',
            'Peugeot 206', 'Peugeot 207', 'Peugeot 208', 'Peugeot 307','Peugeot 308', 'Peugeot 407', 'Peugeot 408',
            'Fiat cronos', 'Fiat argos', 'Fiat toro', 'Fiat palio', 'Fiat siena', 'Fiat uno',                   
            'Ford fiesta', 'Ford Ecosport', 'Ford ka', 'Ford ranger', 'Ford focus',
            'Toyota etios', 'Toyota hilux', 'Toyota corolla', 'Toyota yaris', 
            'Honda civic' ]

#CODIFICACION DE NOMBRES
cars_codif = []
for car in cars:
    cars_codif.append(quote(car))

Con la finalidad de realizar una request por cada modelo de automovil en la lista, se crea la funcion **get_publications()**. Esta funcion  devuelve una lista de diccionarios, cada uno de estos diccionarios representa una publicacion de **MercadoLibre**. Dentro de la logica de la funcion se establece un maximo de 250 publicaciones por vehiculo.

In [11]:
#FUNCION RECURSIVA PARA OBTENER LAS PUBLICACIONES DE CADA VEHICULO
def get_publications(car:str, offset:int, publications_list:list) -> list:
    url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}&offset={offset}'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f'Codigo de estado: {response.status_code}, en el vehiculo: {car}')    
    publications = response.json()
    if publications['results']:
        publications_list.extend(publication for publication in publications['results'])
    if publications['results'] and offset < 400:
        return get_publications(car, offset+50, publications_list)          
    return publications_list

Se crea la lista **publications** la cual alojará los diccionarios obtenidos de la funcion **get_publications()**. Acto seguido ejecuta un ciclo for a fines de recorrer la lista **cars_codif** y ejecutar la funcion **get_publications()** por cada uno de los elementos en la lista, luego los resultados obtenidos se guardan en las lista **publications**

In [12]:
#LISTA VACIA PARA GUARDAR LAS PUBLICACIONES OBTENIDAS
publications = []

#SE OBTIENEN LAS PUBLICACIONES DE LOS MODELOS DE VEHICULOS SELECCIONADOS
for car in cars_codif:
    try:
        publications.extend(get_publications(car, 0, []))
    except Exception as ex:
        print(ex)

#CANTIDAD DE PUBLICACIONES OBTENIDAS
print(f'La cantidad de publicaciones obtenidas es: {len(publications)}')

La cantidad de publicaciones obtenidas es: 16977


Se guarda un archivo **JSON** con el resultado obtenido en el paso anterior

In [13]:
#GUARDADO DE JSON
with open('MeliDataPubl.json','w') as file:
    json.dump(publications, file, indent=4, separators=(',',':'))

#### Extraccion de los datos de interes

Con la finalidad de estructurar los datos extraidos de la API anteriormente se crea la clase **CarPublicaction()** la cual aloja en sus atributos los datos de interes de las publicaciones que se obtubieron. Tambien se crea la clase **CarPublicationBuilder()** cuya responsabilidad es extraer de un diccionario(publicacion) los datos de interes y devolver una instancia de la clase **CarPublicaction()** con sus atributos seteados

In [14]:
class CarPublication():
    def __init__(self):
        self.id = None
        self.category_id = None
        self.title = None
        self.condition = None
        self.car_year = None
        self.brand = None
        self.model = None
        self.version = None
        self.engine = None
        self.engine_power = None
        self.doors = None        
        self.km = None
        self.fuel_type = None        
        self.traction_control = None
        self.passenger_capacity = None
        self.transmission = None
        self.currency = None
        self.price = None
        self.seller_id = None
        self.seller_nickname = None
        self.is_car_shop = None
        self.seller_country = None
        self.seller_state = None
        self.seller_city = None
        self.seller_neighborhood = None

In [15]:
class CarPublicationBuilder():
    def __init__(self, publication:dict):
         self.publication = publication

    def build(self):
        car_publication = CarPublication()
        car_publication.id = self.__get_id()
        car_publication.category_id = self.__get_category_id()
        car_publication.title = self.__get_title()
        car_publication.condition = self.__get_condition()
        car_publication.car_year = self.__get_car_year()
        car_publication.brand = self.__get_brand()
        car_publication.model = self.__get_model()
        car_publication.version = self.__get_version()
        car_publication.engine = self.__get_engine()
        car_publication.engine_power = self.__get_engine_power()
        car_publication.doors = self.__get_doors()
        car_publication.km = self.__get_km()
        car_publication.fuel_type = self.__get_fuel_type()
        car_publication.traction_control = self.__get_traction_control()
        car_publication.passenger_capacity = self.__get_passenger_capacity()
        car_publication.transmission = self.__get_transmission()
        car_publication.currency = self.__get_currency()
        car_publication.price = self.__get_price()
        car_publication.seller_id = self.__get_seller_id()
        car_publication.seller_nickname = self.__get_seller_nickname()
        car_publication.is_car_shop = self.__get_is_oficial_store()
        car_publication.seller_country = self.__get_seller_country()
        car_publication.seller_state = self.__get_seller_state()
        car_publication.seller_city = self.__get_seller_city()
        car_publication.seller_neighborhood = self.__get_seller_neighborhood()
        return car_publication

    def __get_id(self):
        return self.publication.get('id', None)
    
    def __get_category_id(self):
        return self.publication.get('category_id', None)
    
    def __get_title(self):
        return self.publication.get('title', None)
    
    def __get_condition(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'ITEM_CONDITION'):
                return item['value_name']
        return None
    
    def __get_car_year(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'VEHICLE_YEAR'):
                return item['value_name']
        return None
    
    def __get_brand(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'BRAND'):
                return item['value_name']
        return None
    
    def __get_model(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'MODEL'):
                return item['value_name']
        return None
    
    def __get_version(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'TRIM'):
                return item['value_name']
        return None
    
    def __get_engine(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'ENGINE'):
                return item['value_name']
        return None
    
    def __get_engine_power(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'POWER'):
                return item['value_name']
        return None
    
    def __get_doors(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'DOORS'):
                return item['value_name']
        return None
    
    def __get_km(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'KILOMETERS'):
                return item['value_name']
        return None
    
    def __get_fuel_type(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'FUEL_TYPE'):
                return item['value_name']
        return None
    
    def __get_traction_control(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'TRACTION_CONTROL'):
                return item['value_name']
        return None
    
    def __get_passenger_capacity(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'PASSENGER_CAPACITY'):
                return item['value_name']
        return None
    
    def __get_transmission(self):
        for item in self.publication['attributes']:
            if ('id' in item) and (item['id'] == 'TRANSMISSION'):
                return item['value_name']
        return None
    
    def __get_currency(self):
        return self.publication.get('currency_id', None)
    
    def __get_price(self):
        return self.publication.get('price', None)
    
    def __get_seller_id(self):
        return self.publication.get('seller', {}).get('id', None)
    
    def __get_seller_nickname(self):
        return self.publication.get('seller', {}).get('nickname', None)
    
    def __get_is_oficial_store(self):
        return self.publication.get('official_store_id', None) is not None
    
    def __get_seller_country(self):
        return self.publication.get('location',{}).get('country', {}).get('name', None)
    
    def __get_seller_state(self):
        return self.publication.get('location',{}).get('state', {}).get('name', None)
    
    def __get_seller_city(self):
        return self.publication.get('location',{}).get('city', {}).get('name', None)
    
    def __get_seller_neighborhood(self):
        return self.publication.get('location',{}).get('neighborhood', {}).get('name', None)

Mediante un ciclo for se recorre **publications_list** con la finalidad de ejecutar una instancia de la clase **CarPublicationBuilder()** y obtener una objeto de tipo **CarPublicaction()** por cada diccionario en **publications** 

In [16]:
#CICLO PARA RECORRER LAS PUBLICACIONES OBTENIDAS Y EXTRAER LA INFIORMACION DE INTERES
car_publications = [
    CarPublicationBuilder(publication).build()
    for publication in publications
]

#CANTIDAD DE PUBLICACIONES EXTRAIDAS
print(f'Cantidad de registros obtenidos: {len(car_publications)}')

Cantidad de registros obtenidos: 16977


Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [17]:
#CICLO PARA COVERTIR LAS CLASES EN DICCIONARIOS
car_publications = [vars(publication) for publication in car_publications]

with open('MeliCarPublications.json','w' ) as file:
    json.dump(car_publications, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**car_publications**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [18]:
df = pd.DataFrame(car_publications)
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1839176362,MLA1744,Volkswagen Gol,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,6200.0,158350950,MATIASLABASTIE,False,Argentina,Bs.As. G.B.A. Oeste,Moreno,San Patricio
1,MLA1436051759,MLA1744,Volkswagen Gol Power 1.9 Diesel 2008,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 5 p,1.4,83 hp,...,Manual,USD,5200.0,790635263,LAUTARO KARA,False,Argentina,Capital Federal,Capital Federal,Nueva Pompeya
2,MLA1837597104,MLA1744,Volkswagen Gol 1.6 3p Comfortline Ln Ab 2010,Usado,2010,Volkswagen,Gol,1.6 I Power 701,1.6,92 hp,...,Manual,USD,5500.0,239208163,XANGO AUTOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Villa Martelli


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [19]:
df = pd.read_json('MeliCarPublications.json')
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1839176362,MLA1744,Volkswagen Gol,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,6200.0,158350950,MATIASLABASTIE,False,Argentina,Bs.As. G.B.A. Oeste,Moreno,San Patricio
1,MLA1436051759,MLA1744,Volkswagen Gol Power 1.9 Diesel 2008,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 5 p,1.4,83 hp,...,Manual,USD,5200.0,790635263,LAUTARO KARA,False,Argentina,Capital Federal,Capital Federal,Nueva Pompeya
2,MLA1837597104,MLA1744,Volkswagen Gol 1.6 3p Comfortline Ln Ab 2010,Usado,2010,Volkswagen,Gol,1.6 I Power 701,1.6,92 hp,...,Manual,USD,5500.0,239208163,XANGO AUTOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Villa Martelli


In [20]:
print(f'Cantidad de filas y columnas: {df.shape}\n\n')

df.info()

Cantidad de filas y columnas: (16977, 25)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16977 entries, 0 to 16976
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   16977 non-null  object 
 1   category_id          16977 non-null  object 
 2   title                16977 non-null  object 
 3   condition            16977 non-null  object 
 4   car_year             16977 non-null  int64  
 5   brand                16977 non-null  object 
 6   model                16977 non-null  object 
 7   version              16977 non-null  object 
 8   engine               16641 non-null  object 
 9   engine_power         15960 non-null  object 
 10  doors                16977 non-null  int64  
 11  km                   16977 non-null  object 
 12  fuel_type            16977 non-null  object 
 13  traction_control     16517 non-null  object 
 14  passenger_capacity   15785 non-null  float